In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import re
import string

ModuleNotFoundError: No module named 'tensorflow'

In [5]:
bucket_name = 'cloud-ai-platform-2e11e97a-0212-426c-b0fa-b1f124dd9ef2'
file_name = 'NLP_data_v2.json'
from google.cloud import storage
import json

# Instantiate a Google Cloud Storage client and specify required bucket and file
storage_client = storage.Client()
bucket = storage_client.get_bucket(bucket_name)
blob = bucket.blob(file_name)

# Download the contents of the blob as a string and then parse it using json.loads() method
data = json.loads(blob.download_as_string(client=None))

In [6]:
df = pd.DataFrame(data)

In [7]:
import multiprocessing

num_processors = multiprocessing.cpu_count()
print(f'Available CPUs: {num_processors}')

Available CPUs: 4


In [8]:
from pandarallel import pandarallel
pandarallel.initialize(nb_workers=num_processors-1, use_memory_fs=False)

INFO: Pandarallel will run on 3 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [9]:
df_sample = df.sample(1000)

In [2]:
import spacy 
# from spacy import displacy

2023-05-16 16:26:08.056577: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-16 16:26:08.058356: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-05-16 16:26:08.058433: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [4]:
spacy.require_gpu()
nlp = spacy.load("en_core_web_md")

In [11]:
nlp.pipe_names

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

In [19]:
df_sample.head(2)

,url,date,language,title,text,clean_text
95550,https://bisouv.com/uncategorized/4654725/covid...,1618876800000,en,Covid-19 Impact on Artificial Intelligence Har...,\n\nCovid-19 Impact on Artificial Intelligence...,Covid Impact Artificial Intelligence Hardware ...
77792,https://royalsuttonnews.uk/uncategorised/70695...,1604448000000,en,Future of Artificial Intelligence-based Securi...,\nFuture of Artificial Intelligence-based Secu...,Future Artificial Security Market Study Royal ...


In [22]:
%%time

entities = []
labels = []
doc_num = []
for i, row in df.iterrows():
    doc_num.append(i)
    for doc in nlp.pipe(
        texts=[row['clean_text']],
        disable=["tok2vec","parser","attribute_ruler","lemmatizer"]):
        # Extract the named entities and their labels
        ents = []
        lbls = []
        for ent in doc.ents:
            ents.append(ent.text)
            lbls.append(ent.label_)
        entities.append(ents)
        labels.append(lbls)



CPU times: user 1h 29min 38s, sys: 32.5 s, total: 1h 30min 10s
Wall time: 1h 30min 11s


In [31]:
entities_labels = list(zip(doc_num,entities, labels))
entities_labels_df = pd.DataFrame(entities_labels)

entities_labels_df.columns = ["Index","Entities", "Labels"]
# explode the 'Entities' and 'Labels' columns
entities_labels_df = entities_labels_df.explode(['Entities', 'Labels'])

# reset the index to a regular column
entities_labels_df = entities_labels_df.reset_index()

# rename the old index column to 'Index'
entities_labels_df = entities_labels_df.rename(columns={'index': 'Index'})

# reorder the columns to match the original order
entities_labels_df = entities_labels_df[['Index', 'Entities', 'Labels']]
entities_labels_df = entities_labels_df.loc[:,~entities_labels_df.columns.duplicated()]
entities_labels_df[entities_labels_df.Labels == "ORG"]["Entities"].value_counts().reset_index().rename(columns={"index": "Entities", "Entities": "Count"}).head(50)



,Entities,Count
0,AI,180788
1,Google,72166
2,Gray Media Group,62871
3,Microsoft,56062
4,Station Gray Television,31261
5,ChatGPT,27627
6,GPT,22901
7,IBM,22694
8,Artificial Intelligence,19668
9,Amazon,18231


In [30]:
entities_labels_df.Labels.unique()

array(['NORP', 'ORG', 'DATE', 'GPE', 'PERSON', 'TIME', 'CARDINAL', 'LOC',
       'PRODUCT', 'WORK_OF_ART', 'ORDINAL', 'MONEY', 'PERCENT', 'FAC',
       'EVENT', 'QUANTITY', 'LAW', 'LANGUAGE', nan], dtype=object)

In [40]:
entities_labels_df.to_csv('entities_labels.csv',index=True)

In [39]:
# !gsutil cp entities_labels_df gs://cloud-ai-platform-2e11e97a-0212-426c-b0fa-b1f124dd9ef2

CommandException: No URLs matched: entities_labels_df


In [41]:
#entities_labels_df.to_csv('entities_labels.csv', index=False)

# copy the file to GCS bucket using gsutil
!gsutil cp entities_labels.csv gs://cloud-ai-platform-2e11e97a-0212-426c-b0fa-b1f124dd9ef2/

Copying file://entities_labels.csv [Content-Type=text/csv]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

/ [1 files][373.0 MiB/373.0 MiB]                                                
Operation completed over 1 objects/373.0 MiB.                                    


In [3]:
import pandas as pd
df = pd.read_csv('entities_labels.csv')

In [7]:
res = df[df.Labels == "ORG"]["Entities"].value_counts().reset_index().rename(columns={"index": "Entities", "Entities": "Count"})


,Entities,Count
0,AI,180788
1,Google,72166
2,Gray Media Group,62871
3,Microsoft,56062
4,Station Gray Television,31261
...,...,...
259988,VideoThree,2
259989,speakers155 university,2
259990,Sequoia Financial Advisors,2
259991,SparkFun OpenLog Artemis,2


In [8]:
res = res[res.Count > 10]

In [16]:
res.iloc[130:190]

,Entities,Count
130,FacebookShare,1876
131,LinkedIn,1875
132,CloudCision IRAll,1875
133,IPO,1871
134,Decentralized Machine Learning,1865
135,NHS,1859
136,Microsoft Google,1849
137,Artificial Intelligence Service,1839
138,Quantum,1803
139,Telix,1797


### POS Tag

In [31]:
df_sample = df.sample(100)

In [152]:
nlp = spacy.load("en_core_web_sm")

In [153]:
%%time

pos_tags = []
doc_num = []
pos = []
words = []

# disable everything except the tagger
# nlp.disable_pipes(['tok2vec', 'senter', 'attribute_ruler', 'lemmatizer'])

for i, row in df_sample[['clean_text']].iterrows():
    # Extract the string value from the clean_text column
    text = row['clean_text']
    
    doc = nlp(text)
    for token in doc:
        words.append(token)
        doc_num.append(i)
        # Extract the POS tags
        pos.append(token.pos_)
        pos_tags.append(token.tag_)



In [154]:
x = pd.DataFrame({'Doc':doc_num,'Word':words,'POS_Tags':pos_tags, 'POS': pos})

In [158]:
x[x.POS=='PROPN']

,Doc,Word,POS_Tags,POS
3,171863,AI,NNP,PROPN
15,171863,Zhu,NNP,PROPN
16,171863,Ye,NNP,PROPN
17,171863,Sign,NNP,PROPN
18,171863,inAbout,NNP,PROPN
...,...,...,...,...
79966,172561,Media,NNP,PROPN
79967,172561,Group,NNP,PROPN
79969,172561,Station,NNP,PROPN
79970,172561,Gray,NNP,PROPN
